## Importing the data source

In [11]:
#import libraries
import pandas as pd
import csv
import sqlite3
import os
import matplotlib.pyplot as plt

os.getcwd()
#save current pathfile to a variable and prints current pathfile
pathfile = os.getcwd()
pathfile += "\\"

print(pathfile)

#import csv file
df = pd.read_csv("data/businesses.csv")
df2 = pd.read_csv("data/inspections.csv")
df3 = pd.read_csv("data/violations.csv")

C:\Users\amahm\Documents\Classes\CodeLouisville\Python\Notebooks\


## Converting data source into a sql table

In [12]:
# create the sqlite DB
db = sqlite3.connect("data/resturants.db")

# convert the pandas dataframe to a sqlite table
df.to_sql("business", db, if_exists="replace")
df2.to_sql("inspections", db, if_exists="replace")
df3.to_sql("violations", db, if_exists="replace") #unusable due to error in business_id

In [13]:
#view one sql table before merging
sql='''
SELECT * 
FROM violations
LIMIT 5;
'''

pd.read_sql_query(sql,db)

,index,business_id,date,code,description
0,0,29790,20181212,None,Food/non-food contact surfaces not approved or...
1,1,29790,20181212,None,Non-food contact surfaces of equipment/utensil...
2,2,29790,20181212,None,"Wall/Ceiling improperly constructed, not in go..."
3,3,29790,20181212,None,"Floor improperly constructed, not draining, no..."
4,4,29790,20180709,None,"Floor improperly constructed, not draining, no..."


In [14]:
#view merged sql table
sql='''
SELECT *
FROM business AS bus

LEFT OUTER JOIN inspections AS ins 
ON bus.business_id = ins.business_id
;
'''

pd.read_sql_query(sql, db)

,index,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,index,business_id,score,date,description,type
0,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,1915.0,87593.0,99.0,20110228.0,None,routine
1,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,3121.0,87593.0,96.0,20110830.0,None,routine
2,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,4260.0,87593.0,96.0,20120117.0,None,routine
3,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,5593.0,87593.0,96.0,20120614.0,None,routine
4,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,7317.0,87593.0,96.0,20121219.0,None,routine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35024,2421,105428,J GUMBOS,2109 FRANKFORT AVE,LOUISVILLE,KY,40206,38.254099,-85.707983,(502) 896-4046,34403.0,105428.0,100.0,20190128.0,None,routine
35025,2422,106853,DAVE AND BUSTERS,5000 SHELBYVILLE RD,LOUISVILLE,KY,40207,38.250261,-85.624661,(502) 963-0940,NaN,NaN,NaN,NaN,None,None
35026,2423,106862,TOWNEPLACE SUITES,6601 PARAMOUNT PARK DR,LOUISVILLE,KY,40213,0.000000,0.000000,(502) 749-6634,NaN,NaN,NaN,NaN,None,None
35027,2424,105478,DOUBLE FLAME BURGERS,5408 VALLEY STATION RD,LOUISVILLE,KY,40272,38.106566,-85.863673,(502) 821-0857,31532.0,105478.0,98.0,20180712.0,None,routine


## Creating a new sql table by combining two datasets

In [20]:
#create merged sql table
#create cursor object first
cursorObj = db.cursor()

#create table
#ISSUE: why doesn't it save the newly formatted field stringdate?????
create_tableName = "mergedTable"

new_tableSQL = f'''
CREATE TABLE if not exists {create_tableName} AS
SELECT *
FROM business AS bus

LEFT OUTER JOIN inspections AS ins 
ON bus.business_id = ins.business_id
;
'''

cursorObj.execute(new_tableSQL)

cursorObj.close()

In [21]:
#view newly created sql table
sql ='''
SELECT *
FROM mergedTable;
'''

pd.read_sql_query(sql,db)

,index,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,index:1,business_id:1,score,date,description,type
0,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,1915.0,87593.0,99.0,20110228.0,None,routine
1,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,3121.0,87593.0,96.0,20110830.0,None,routine
2,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,4260.0,87593.0,96.0,20120117.0,None,routine
3,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,5593.0,87593.0,96.0,20120614.0,None,routine
4,0,87593,SICILIAN PIZZA AND PASTA,627 S 4TH ST,LOUISVILLE,KY,40202,38.247844,-85.758056,(502) 457-3206,7317.0,87593.0,96.0,20121219.0,None,routine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35024,2421,105428,J GUMBOS,2109 FRANKFORT AVE,LOUISVILLE,KY,40206,38.254099,-85.707983,(502) 896-4046,34403.0,105428.0,100.0,20190128.0,None,routine
35025,2422,106853,DAVE AND BUSTERS,5000 SHELBYVILLE RD,LOUISVILLE,KY,40207,38.250261,-85.624661,(502) 963-0940,NaN,NaN,NaN,NaN,None,None
35026,2423,106862,TOWNEPLACE SUITES,6601 PARAMOUNT PARK DR,LOUISVILLE,KY,40213,0.000000,0.000000,(502) 749-6634,NaN,NaN,NaN,NaN,None,None
35027,2424,105478,DOUBLE FLAME BURGERS,5408 VALLEY STATION RD,LOUISVILLE,KY,40272,38.106566,-85.863673,(502) 821-0857,31532.0,105478.0,98.0,20180712.0,None,routine


In [22]:
#review datatypes for mergedTable
query_details='''
SELECT * 
FROM PRAGMA_TABLE_INFO('mergedTable');
'''

pd.read_sql_query(query_details,db)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INT,0,None,0
1,1,business_id,INT,0,None,0
2,2,name,TEXT,0,None,0
3,3,address,TEXT,0,None,0
4,4,city,TEXT,0,None,0
5,5,state,TEXT,0,None,0
6,6,postal_code,INT,0,None,0
7,7,latitude,REAL,0,None,0
8,8,longitude,REAL,0,None,0
9,9,phone_number,TEXT,0,None,0


## Cleaning the data

In [51]:
#Converting date column datatype from integer to datetime
sql_edited ='''
SELECT SUBSTR(CAST(date as text), 0,5)||'-'
|| SUBSTR(CAST(date as text), 5,2)||'-'
|| SUBSTR(CAST(date as text), 7,2) AS DATE_FORMATTED
FROM mergedTable;
'''

pd.read_sql_query(sql_edited,db)

,DATE_FORMATTED
0,2011-02-28
1,2011-08-30
2,2012-01-17
3,2012-06-14
4,2012-12-19
...,...
35024,2019-01-28
35025,None
35026,None
35027,2018-07-12


In [61]:
#aggregate the score data
sql='''
SELECT name, AVG(score) AS Average_Audit_Score
FROM mergedTable
GROUP BY name
LIMIT 10;
'''

pd.read_sql_query(sql,db)

,name,Average_Audit_Score
0,#1 CHINESE RESTAURANT,96.666667
1,#52 SPORTS BAR AND KITCHEN,97.411765
2,1ST WOK CHINESE RESTAURANT,92.347826
3,211 CLOVER LANE,91.500000
4,4 KINGS CAFE,84.263158
5,4 SEASONS RESTAURANT,100.000000
6,40 ACRES AND A MULE,94.000000
7,4TH AVE UNITED METHODIST CHURCH,94.727273
8,502 BAR AND BISTRO,98.666667
9,610 MAGNOLIA,95.823529
